 ## Question
By given a random array of integer, find all the combinations for any 3 number and sum up equals zero**

ex. given an array [-1,12,0,2,1,3,-4]
will get the combination (-1,0,1) (1,3,-4)

## **Define a timer to count execution time for code block**

In [0]:
from time import time, strftime, localtime
from datetime import timedelta

class CodeTimer(object):

    recentlog = ""

    def __init__(self, codesnip='== default wrap codes ==', **kwargs):
        self.codesnip = codesnip
        super().__init__(**kwargs)

    def __enter__(self):
        self.start = time()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.__endlog()


    def __endlog(self):
        self.end = time()
        self.elapsed = self.end - self.start

        CodeTimer.recentlog = "[ {} ] executed in {}".format(self.codesnip, self.__secondsToStr(self.elapsed))
        print(CodeTimer.recentlog)

    def __secondsToStr(self, elapsed):
        return str(timedelta(seconds=elapsed))

## 1. Start from a simple loop to get all combinations then check sum one by one

In [0]:
def sum_to_zero_by_loop(lst):
    # to grab 3 elements from the list also walk through all the possibility
    combs = []
    with CodeTimer('find all combinations'):
        # try a for loop
        for i in range(len(lst)):
            for j in range(i+1, len(lst)):
                for k in range(j+1, len(lst)):
                    combs.append([lst[i],lst[j],lst[k]])

    
    # loop the combination and sum
    with CodeTimer('find all groups with sum==0'):
        zerosumlist = [ cb for cb in combs if sum(cb)==0 ]

    return zerosumlist

### for a small array, there is no performance concern

In [3]:
%%time
rlt = sum_to_zero_by_loop([-1,12,0,2,1,3,-4,8,22,-19,5,-14])
print(rlt)

[ find all combinations ] executed in 0:00:00.000078
[ find all groups with sum==0 ] executed in 0:00:00.000057
[[-1, 0, 1], [-1, -4, 5], [12, 2, -14], [1, 3, -4]]
CPU times: user 953 µs, sys: 989 µs, total: 1.94 ms
Wall time: 1.34 ms


### or a lager array (500), obviously it's not efficient (more than 30 sec)

In [5]:
import numpy as np
def find_all_combination_by_loop(cnt):
    samples = np.random.RandomState(0).randint(low=-10000, high=10000, size=cnt)
    rlt = sum_to_zero_by_loop(samples)
    print('find {} groups which sum==0'.format(len(rlt)))

find_all_combination_by_loop(500)

[ find all combinations ] executed in 0:00:20.583952
[ find all groups with sum==0 ] executed in 0:00:10.079492
find 725 groups which sum==0


## 2. Use itertools to get combination instead of a loop

In [0]:
import itertools
def sum_to_zero_by_iter(lst):
    # to grab 3 elements from the list also walk through all the possibility

    with CodeTimer('find all combinations by itertools'):
        combs = itertools.combinations(lst,3)

    # loop the combination and sum
    with CodeTimer('find all groups with sum==0'):
        zerosumlist = [ cb for cb in combs if sum(cb)==0 ]

    return zerosumlist

def find_all_combination_by_iter(cnt):
    samples = np.random.RandomState(0).randint(low=-10000, high=10000, size=cnt)
    rlt = sum_to_zero_by_iter(samples)
    print('find {} groups which sum==0'.format(len(rlt)))

### itertools can get all the combinations within no time, the exection time is all for checking sum

In [8]:
find_all_combination_by_iter(500)

[ find all combinations by itertools ] executed in 0:00:00.000044
[ find all groups with sum==0 ] executed in 0:00:10.603627
find 725 groups which sum==0


## 3. Try to check the sum in loop directly

In [0]:
def sum_to_zero_by_onestep_loop(lst):
    # to grab 3 elements from the list also walk through all the possibility
    zerosumlist = []
    with CodeTimer('find all group in one step'):
        # try a for loop
        for i in range(len(lst)):
            for j in range(i+1, len(lst)):
                for k in range(j+1, len(lst)):
                    if ((lst[i] + lst[j] + lst[k]) == 0):
                        zerosumlist.append([lst[i],lst[j],lst[k]])
    
    return zerosumlist

def find_all_combination_by_onestep_loop(cnt):
    samples = np.random.RandomState(0).randint(low=-10000, high=10000, size=cnt)
    rlt = sum_to_zero_by_onestep_loop(samples)
    print('find {} groups which sum==0'.format(len(rlt)))

### the same size array (500) use roughly the same if check the sum in the loop

In [10]:
find_all_combination_by_onestep_loop(500)

[ find all group in one step ] executed in 0:00:11.084332
find 725 groups which sum==0


### <font color='blue'>the looping function above actually is the same as chcking sum on the combination, which causes the slowness</font>

## 4. Try sort the array before looping
### and put 2 more checkings based on the sorted array

In [0]:
def sum_to_zero_by_sort_and_loop(lst):
    # to grab 3 elements from the list also walk through all the possibility
    zerosumlist = []
    with CodeTimer('sort the array'):
        srt = np.sort(lst)
        
    with CodeTimer('find all group on sorted array'):
        # try a for loop
        for i in range(len(srt)):
            for j in range(i+1, len(lst)):
                tmp = srt[i] + srt[j]
                # check #1: 
                # if the sum of 2 number already larger than 0, then stop
                if (tmp > 0):
                    break
                else:
                    # check #2:
                    # loop from the end of sorted array, stop the loop if the sum is less than 0
                    for k in range(len(srt)-1, j, -1):
                        if ((tmp + srt[k]) == 0):
                            zerosumlist.append([srt[i],srt[j],srt[k]])
                        elif ((tmp +srt[k]) < 0):
                            break
    
    return zerosumlist

def find_all_combination_by_sort_and_loop(cnt):
    samples = np.random.RandomState(0).randint(low=-10000, high=10000, size=cnt)
    rlt = sum_to_zero_by_sort_and_loop(samples)
    print('find {} groups which sum==0'.format(len(rlt)))

### the sorting takes no time, and the final result takes less than 4 seconds now

In [12]:
find_all_combination_by_sort_and_loop(500)

[ sort the array ] executed in 0:00:00.001152
[ find all group on sorted array ] executed in 0:00:03.347215
find 725 groups which sum==0
